In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [ ]:
excel_file = "salesdaily-data.xlsx"
sheet_name = "Worksheet"
column_name = "J05"

df = pd.read_excel(excel_file, sheet_name=sheet_name)
DATA=df.drop(columns=['Year','Hour',])
df.index = pd.to_datetime(df['datetime'], format='%m/%d/%Y')
med1 = df['J05']
med1.plot()
def df_to_X_y(df, window_size=5):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [[a] for a in df_as_np[i:i+window_size]]
    X.append(row)
    label = df_as_np[i+window_size]
    y.append(label)
  return np.array(X), np.array(y)


In [ ]:
def df_to_X_y(df, window_size=5):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [[a] for a in df_as_np[i:i+window_size]]
    X.append(row)
    label = df_as_np[i+window_size]
    y.append(label)
  return np.array(X), np.array(y)
WINDOW_SIZE = 5
X1, y1 = df_to_X_y(med1, WINDOW_SIZE)
X1.shape, y1.shape
X_train1, y_train1 = X1[:1500], y1[:1500]
X_val1, y_val1 = X1[1500:1900], y1[1500:1900]
X_test1, y_test1 = X1[1900:], y1[1900:]
X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

model1 = Sequential()
model1.add(InputLayer((5, 1)))
model1.add(LSTM(64))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear'))

model1.summary()
cp1 = ModelCheckpoint('model1/model1.keras', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model1.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=100, callbacks=[cp1])
from tensorflow.keras.models import load_model
model1 = load_model('model1/model1.keras')
train_predictions = model1.predict(X_train1).flatten()
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y_train1})
train_results
plt.figure(figsize=(12, 6))
plt.plot(train_results['Train Predictions'][5:400], label='Train Predictions')
plt.plot(train_results['Actuals'][5:400], label='Actuals')
plt.legend()
plt.title('Train Predictions vs Actuals')
plt.xlabel('Index')
plt.ylabel('Value')
plt.show()

# Generating validation predictions
val_predictions = model1.predict(X_val1).flatten()
val_results = pd.DataFrame(data={'Val Predictions': val_predictions, 'Actuals': y_val1})

# Plotting validation results
plt.figure(figsize=(12, 6))
plt.plot(val_results['Val Predictions'][:400], label='Val Predictions')
plt.plot(val_results['Actuals'][:400], label='Actuals')
plt.legend()
plt.title('Validation Predictions vs Actuals')
plt.xlabel('Index')
plt.ylabel('Value')
plt.show()
#generating test result
test_predictions = model1.predict(X_test1).flatten()
test_results = pd.DataFrame(data={'Test Predictions':test_predictions, 'Actuals':y_test1})
test_results

# Plotting test results
plt.figure(figsize=(12, 6))
plt.plot(test_results['Test Predictions'][:400], label='test Predictions')
plt.plot(test_results['Actuals'][:400], label='Actuals')
plt.legend()
plt.title('test Predictions vs Actuals')
plt.xlabel('Index')
plt.ylabel('Value')
plt.show()
zero_mask = y_train1.flatten() == 0

# Calculate MAPE
mape = np.mean(np.abs((y_test1 - test_predictions) / y_test1)) * 100
print(f'MAPE: {mape}')
results = pd.DataFrame({'Actual': y_test1, 'Predicted': test_predictions})
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test1, test_predictions))
print(f'Test RMSE: {rmse}')
